# Writing CUDA kernels in Python

In this session we learn about how we can write simple kernels in python, translate them into C++ code using an LLM and leverage pytorch's tool to compile the kernel into a module we can load and execute from python.

Video link: https://www.youtube.com/watch?v=4sgKnKbR-WE

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/b4rdos/gpu-mode-notes/blob/master/notebooks/03-kernels-in-pythono.ipynb)